In [3]:
import warnings
import math
import datetime
import time
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

#### **Import DRIAS-CLIMAT** 
- Production : eurocordex2020
- Modele : CNRM-ALADIN63 - CNRM-CERFACS-CNRM-CM5 (RCM - GCM)
- Scenario : rcp4.5 / rcp2.5
- Donnees CORRIGEES
- Periode : 01/01/2006 - 31/12/2100
    1. <span style="color: #7369FFFF;">Format de la ligne</span>  : "Date Latitude Longitude  tasminAdjust tasmaxAdjust tasAdjust prtotAdjust prsnAdjust
    2. <span style="color: #7369FFFF;">Date</span> : Date du jour sous la forme 'JJ/MM/AAAA'
    3. <span style="color: #7369FFFF;">Latitude Longitude</span> : position du point de grille SAFRAN en degres decimaux (WGS84)
    4. <span style="color: #7369FFFF;">tasminAdjust</span> : Temperature minimale journaliere a 2 m [K]
    5. <span style="color: #7369FFFF;">tasmaxAdjust</span> : Temperature maximale journaliere a 2 m [K]
    6. <span style="color: #7369FFFF;">tasAdjust</span> : Temperature moyenne a 2 m [K]
    7. <span style="color: #7369FFFF;">prtotAdjust</span> : Precipitations totales [kg/m2/s]
    8. <span style="color: #7369FFFF;">prsnAdjust</span> : Precipitations solides [kg/m2/s]
    9. <span style="color: #7369FFFF;">hussAdjust</span> : Humidite specifique a 2 m [kg/kg]
    10. <span style="color: #7369FFFF;">rsdsAdjust</span> : Rayonnement visible incident a la surface [W/m2]
    11. <span style="color: #7369FFFF;">rldsAdjust</span> : Rayonnement infra-rouge incident a la surface [W/m2]
    12. <span style="color: #7369FFFF;">sfcWindAdjust</span> : Vitesse du vent a 10 m [m/s]
    13. <span style="color: #7369FFFF;">evspsblpotAdjust</span> : Evapo-transpiration potentielle Hargreaves [kg/m2/s]

In [7]:
data_rcp_4_5 = pd.read_excel('CNRM-CERFACS-CNRM-CM5_CNRM-ALADIN63_rcp4.5_METEO-FRANCE_ADAMONT-France_SAFRAN_day_2006_2100.xlsx')
data_rcp_4_5

,Date,Latitude,Longitude,tasminAdjust,tasmaxAdjust,tasAdjust,prtotAdjust,prsnAdjust,hussAdjust,rsdsAdjust,rldsAdjust,sfcWindAdjust,evspsblpotAdjust
0,01/01/2006,49.2427,4.03882,270.00,277.80,273.13,0.000003,0.000003,0.003503,53.157,264.926,2.138,1.800000e-06
1,02/01/2006,49.2427,4.03882,273.19,277.21,275.18,0.000097,0.000012,0.005015,9.143,347.306,7.012,5.000000e-07
2,03/01/2006,49.2427,4.03882,281.74,282.12,281.97,0.000008,0.000000,0.008196,8.756,381.686,8.374,2.000000e-06
3,04/01/2006,49.2427,4.03882,282.91,286.21,285.07,0.000001,0.000000,0.007773,16.775,355.457,5.803,5.500000e-06
4,05/01/2006,49.2427,4.03882,278.30,284.15,280.35,0.000000,0.000000,0.006088,29.116,337.960,1.557,2.200000e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34693,27/12/2100,49.2427,4.03882,274.15,282.63,277.70,0.000000,0.000000,0.004967,51.618,286.619,3.158,2.300000e-06
34694,28/12/2100,49.2427,4.03882,277.29,285.13,281.41,0.000042,0.000000,0.005910,38.051,318.954,4.029,4.300000e-06
34695,29/12/2100,49.2427,4.03882,277.76,282.73,281.21,0.000032,0.000000,0.005180,30.541,324.057,4.353,7.100000e-06
34696,30/12/2100,49.2427,4.03882,279.30,283.26,280.72,0.000108,0.000000,0.005884,17.554,334.634,10.269,8.800000e-06


In [6]:
data_rcp_26 = pd.read_excel('CNRM-CERFACS-CNRM-CM5_CNRM-ALADIN63_rcp2.6_METEO-FRANCE_ADAMONT-France_SAFRAN_day_2006_2100.xlsx')
data_rcp_26

,Date,Latitude,Longitude,tasminAdjust,tasmaxAdjust,tasAdjust,prtotAdjust,prsnAdjust,hussAdjust,rsdsAdjust,rldsAdjust,sfcWindAdjust,evspsblpotAdjust
0,01/01/2006,49.2427,4.03882,270.00,277.74,273.11,0.000000e+00,0.000000,0.003508,53.716,265.013,2.181,1.800000e-06
1,02/01/2006,49.2427,4.03882,273.39,277.19,275.27,1.020000e-04,0.000011,0.004973,9.241,344.004,7.124,6.000000e-07
2,03/01/2006,49.2427,4.03882,281.72,282.36,282.11,8.600000e-06,0.000000,0.008280,7.535,382.930,8.292,1.800000e-06
3,04/01/2006,49.2427,4.03882,283.02,287.32,285.83,5.400000e-06,0.000000,0.008134,19.244,356.590,5.446,5.000000e-06
4,05/01/2006,49.2427,4.03882,280.74,284.66,282.11,0.000000e+00,0.000000,0.006754,14.233,367.906,2.020,3.400000e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34693,27/12/2100,49.2427,4.03882,273.82,277.76,275.78,0.000000e+00,0.000000,0.003508,53.907,263.422,3.435,6.800000e-06
34694,28/12/2100,49.2427,4.03882,272.41,275.68,273.87,1.170000e-05,0.000008,0.003386,12.878,314.804,3.790,5.000000e-06
34695,29/12/2100,49.2427,4.03882,272.05,274.35,272.91,4.400000e-06,0.000004,0.003521,18.212,310.789,1.192,2.100000e-06
34696,30/12/2100,49.2427,4.03882,270.02,273.22,271.65,8.000000e-07,0.000000,0.002961,11.396,312.622,1.625,2.300000e-06
